In [ ]:
pip install selenium requests beautifulsoup4 scrapy pandas yfinance


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape stock data from Screener.in
def scrape_stock_data(symbol):
    try:
        url = f"https://www.screener.in/company/{symbol}/"
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.content, "html.parser")

        # Extracting data
        try:
            pe_tag = soup.find("span", string="P/E")
            eps_tag = soup.find("span", string="EPS")
            high_low_tag = soup.find("span", string="52 Week High/Low")
            market_cap_tag = soup.find("span", string="Market Cap")
            ltp_tag = soup.find("span", string="Current Price")

            data = {
                "Symbol": symbol,
                "PE": pe_tag.find_next("span").text.strip() if pe_tag else None,
                "EPS": eps_tag.find_next("span").text.strip() if eps_tag else None,
                "52 Week High/Low": high_low_tag.find_next("span").text.strip() if high_low_tag else None,
                "Market Cap": market_cap_tag.find_next("span").text.strip() if market_cap_tag else None,
                "LTP": ltp_tag.find_next("span").text.strip() if ltp_tag else None
            }
        except AttributeError:
            data = {
                "Symbol": symbol,
                "PE": None,
                "EPS": None,
                "52 Week High/Low": None,
                "Market Cap": None,
                "LTP": None
            }
        
        return data
    
    except requests.RequestException as e:
        print(f"Error while loading data for {symbol}: {e}")
        return {"Symbol": symbol, "PE": None, "EPS": None, "52 Week High/Low": None, "Market Cap": None, "LTP": None}

# Function to scrape data for multiple stocks
def scrape_nifty50_screener():
    nifty50_symbols = [
        "RELIANCE", "TCS", "INFY", "HDFCBANK", "HINDUNILVR", 
        "ICICIBANK", "KOTAKBANK", "ITC", "LT", "SBIN", 
        "BAJFINANCE", "BHARTIARTL", "ASIANPAINT", "AXISBANK", 
        "MARUTI", "SUNPHARMA", "ULTRACEMCO", "WIPRO", 
        "HCLTECH", "POWERGRID"
    ]
    
    all_stock_data = []
    for symbol in nifty50_symbols:
        print(f"Scraping data for: {symbol}")
        stock_data = scrape_stock_data(symbol)
        all_stock_data.append(stock_data)
    
    # Create a DataFrame
    df = pd.DataFrame(all_stock_data)
    return df

# Execute and display the DataFrame
nifty50_df = scrape_nifty50_screener()
print(nifty50_df)


Scraping data for: RELIANCE
Scraping data for: TCS
Scraping data for: INFY
Scraping data for: HDFCBANK
Scraping data for: HINDUNILVR
Scraping data for: ICICIBANK
Scraping data for: KOTAKBANK
Scraping data for: ITC
Scraping data for: LT
Scraping data for: SBIN
Scraping data for: BAJFINANCE
Scraping data for: BHARTIARTL
Scraping data for: ASIANPAINT
Scraping data for: AXISBANK
Scraping data for: MARUTI
Scraping data for: SUNPHARMA
Scraping data for: ULTRACEMCO
Scraping data for: WIPRO
Scraping data for: HCLTECH
Scraping data for: POWERGRID
        Symbol    PE   EPS 52 Week High/Low Market Cap   LTP
0     RELIANCE  None  None             None       None  None
1          TCS  None  None             None       None  None
2         INFY  None  None             None       None  None
3     HDFCBANK  None  None             None       None  None
4   HINDUNILVR  None  None             None       None  None
5    ICICIBANK  None  None             None       None  None
6    KOTAKBANK  None  None   

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Predefined Nifty50 stock symbols (can be updated if needed)
nifty50_symbols = [
    "RELIANCE", "TCS", "INFY", "HDFCBANK", "HINDUNILVR", "ICICIBANK", "KOTAKBANK", 
    "ITC", "LT", "SBIN", "BAJFINANCE", "BHARTIARTL", "ASIANPAINT", "AXISBANK", 
    "MARUTI", "SUNPHARMA", "ULTRACEMCO", "WIPRO", "HCLTECH", "POWERGRID"
]

# Function to set up Selenium WebDriver
def setup_driver():
    service = Service(r"D:\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")  # Replace with your ChromeDriver path
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run headless
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to scrape data for a single stock from Screener.in
def scrape_stock_data(driver, symbol):
    stock_url = f"https://www.screener.in/company/{symbol}/"
    driver.get(stock_url)
    time.sleep(2)  # Wait for page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    data = {"Symbol": symbol}

    # Extract data fields
    try:
        data["PE"] = soup.find("span", string="P/E").find_next("span").text.strip()
        data["EPS"] = soup.find("span", string="EPS").find_next("span").text.strip()
        data["52 Week High/Low"] = soup.find("span", string="52 Week High/Low").find_next("span").text.strip()
        data["Market Cap"] = soup.find("span", string="Market Cap").find_next("span").text.strip()
        data["LTP"] = soup.find("span", string="Current Price").find_next("span").text.strip()
    except AttributeError:
        # Handle missing data
        data["PE"] = data["EPS"] = data["52 Week High/Low"] = data["Market Cap"] = data["LTP"] = None

    return data

# Main function to scrape data for all Nifty50 stocks
def scrape_nifty50_screener():
    driver = setup_driver()
    all_stock_data = []
    
    try:
        for symbol in nifty50_symbols:
            print(f"Scraping data for: {symbol}")
            stock_data = scrape_stock_data(driver, symbol)
            all_stock_data.append(stock_data)
    finally:
        driver.quit()
    
    # Convert to DataFrame
    df = pd.DataFrame(all_stock_data)
    return df

# Execute and display the DataFrame
nifty50_df = scrape_nifty50_screener()
print(nifty50_df)

Scraping data for: RELIANCE
Scraping data for: TCS
Scraping data for: INFY
Scraping data for: HDFCBANK
Scraping data for: HINDUNILVR
Scraping data for: ICICIBANK
Scraping data for: KOTAKBANK
Scraping data for: ITC
Scraping data for: LT
Scraping data for: SBIN
Scraping data for: BAJFINANCE
Scraping data for: BHARTIARTL
Scraping data for: ASIANPAINT
Scraping data for: AXISBANK
Scraping data for: MARUTI
Scraping data for: SUNPHARMA
Scraping data for: ULTRACEMCO
Scraping data for: WIPRO
Scraping data for: HCLTECH
Scraping data for: POWERGRID
        Symbol    PE   EPS 52 Week High/Low Market Cap   LTP
0     RELIANCE  None  None             None       None  None
1          TCS  None  None             None       None  None
2         INFY  None  None             None       None  None
3     HDFCBANK  None  None             None       None  None
4   HINDUNILVR  None  None             None       None  None
5    ICICIBANK  None  None             None       None  None
6    KOTAKBANK  None  None   

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf

# Function to scrape data from Screener.in
def scrape_screener_data(symbol):
    try:
        url = f"https://www.screener.in/company/{symbol}/"
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.content, "html.parser")

        # Extracting data
        data = {"Symbol": symbol}
        
        # P/E
        pe_tag = soup.find("span", string="P/E")
        data["PE"] = pe_tag.find_next("span").text.strip() if pe_tag else None
        
        # EPS
        eps_tag = soup.find("span", string="EPS")
        data["EPS"] = eps_tag.find_next("span").text.strip() if eps_tag else None
        
        # 52 Week High/Low
        high_low_tag = soup.find("span", string="52 Week High/Low")
        data["52 Wk High/Low"] = high_low_tag.find_next("span").text.strip() if high_low_tag else None
        
        # Market Cap
        market_cap_tag = soup.find("span", string="Market Cap")
        data["Market Cap"] = market_cap_tag.find_next("span").text.strip() if market_cap_tag else None
        
        # LTP
        ltp_tag = soup.find("span", string="Current Price")
        data["LTP"] = ltp_tag.find_next("span").text.strip() if ltp_tag else None
        
        return data
    
    except requests.RequestException as e:
        print(f"Error while loading data for {symbol}: {e}")
        return {"Symbol": symbol, "PE": None, "EPS": None, "52 Wk High/Low": None, "Market Cap": None, "LTP": None}

# Function to get historical data using Yahoo Finance
def get_historical_data(symbol, period="5y"):
    try:
        stock = yf.Ticker(symbol)
        hist = stock.history(period=period)
        return hist['Close'] if 'Close' in hist else None
    except Exception as e:
        print(f"Error fetching historical data for {symbol}: {e}")
        return None

# Function to calculate returns
def calculate_returns(prices):
    if prices is not None and len(prices) > 0:
        six_month_return = (prices.iloc[-1] / prices.iloc[0]) - 1
        one_year_return = (prices.iloc[-1] / prices.iloc[0]) - 1
        five_year_return = (prices.iloc[-1] / prices.iloc[0]) - 1
        return {
            "6 Month Return": six_month_return,
            "1 Year Return": one_year_return,
            "5 Year Return": five_year_return
        }
    return {"6 Month Return": None, "1 Year Return": None, "5 Year Return": None}

# Function to scrape and get all data for Nifty50 stocks
def scrape_nifty50_data():
    nifty50_symbols = [
        "RELIANCE", "TCS", "INFY", "HDFCBANK", "HINDUNILVR", 
        "ICICIBANK", "KOTAKBANK", "ITC", "LT", "SBIN", 
        "BAJFINANCE", "BHARTIARTL", "ASIANPAINT", "AXISBANK", 
        "MARUTI", "SUNPHARMA", "ULTRACEMCO", "WIPRO", 
        "HCLTECH", "POWERGRID"
    ]
    
    all_stock_data = []
    for symbol in nifty50_symbols:
        print(f"Scraping data for: {symbol}")
        
        # Skip over delisted or unavailable symbols
        ohlcv_data = get_historical_data(symbol, period="5y")
        if ohlcv_data is None:
            print(f"$ {symbol}: possibly delisted; no price data found  (period=5y) (Yahoo error = 'No data found, symbol may be delisted')")
            continue
        
        # Scraping data from Screener
        screener_data = scrape_screener_data(symbol)
        
        # Calculate returns
        returns = calculate_returns(ohlcv_data)
        
        # Merge returns with stock data
        screener_data.update(returns)
        
        all_stock_data.append(screener_data)
    
    # Create a DataFrame
    df = pd.DataFrame(all_stock_data)
    return df

# Execute and display the DataFrame
nifty50_df = scrape_nifty50_data()
print(nifty50_df)


Scraping data for: RELIANCE


$RELIANCE: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


Scraping data for: TCS
Scraping data for: INFY
Scraping data for: HDFCBANK


$HDFCBANK: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


Scraping data for: HINDUNILVR


...

Scraping data for: HCLTECH


$HCLTECH: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


Scraping data for: POWERGRID


$POWERGRID: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")


        Symbol    PE   EPS 52 Wk High/Low Market Cap   LTP  6 Month Return  \
0     RELIANCE  None  None           None       None  None             NaN   
1          TCS  None  None           None       None  None       -0.991500   
2         INFY  None  None           None       None  None        1.613686   
3     HDFCBANK  None  None           None       None  None             NaN   
4   HINDUNILVR  None  None           None       None  None             NaN   
5    ICICIBANK  None  None           None       None  None             NaN   
6    KOTAKBANK  None  None           None       None  None             NaN   
7          ITC  None  None           None       None  None       -0.914199   
8           LT  None  None           None       None  None             NaN   
9         SBIN  None  None           None       None  None             NaN   
10  BAJFINANCE  None  None           None       None  None             NaN   
11  BHARTIARTL  None  None           None       None  None      